In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time 
from pandas import DataFrame

from selenium.webdriver.support.select import Select

In [9]:
driver = webdriver.Chrome('../chromedriver')
driver.get("http://218.94.78.91:18000/#/portal") 
time.sleep(3)
driver.find_elements_by_class_name("Jggs")[1].find_element_by_class_name("menuMore").click()
time.sleep(1)

page_items = driver.find_elements_by_css_selector(".ant-pagination-item.ng-star-inserted")
max_page = page_items[-1].text
print(max_page + " pages in total.")

result = []
header = ["公告名称", "发布时间", "企业名称", '所在区域', '标的','数量','价格','联系人','联系方式']

while True:
    current_page = driver.find_element_by_css_selector(".ant-pagination-item.ng-star-inserted.ant-pagination-item-active").text
    page_items = driver.find_elements_by_css_selector(".ant-pagination-item.ng-star-inserted")
    temp_max_page = page_items[-1].text
    if int(temp_max_page) > int(max_page):
        max_page = temp_max_page
        print('Max page is changed to ' + max_page)
    
    print("---------------------")
    print("current page: " + current_page + '/' + max_page)
    data_list = driver.find_element_by_class_name("PWQ_crgg_cont1").find_elements_by_tag_name("li")

    for data in data_list:
        data.find_element_by_class_name("noties").click()

        table_html = driver.find_element_by_class_name("ant-descriptions-view").find_elements_by_tag_name("td")
        table_data = []
        for item in table_html:
            table_data.append(item.text)
        exclude_count = 0
        for x in header:
            try:
                table_data.remove(x)
            except Exception:
                print('Element not in list')
                exclude_count += 1

        if exclude_count != 0:
            print('This record has ' + str(exclude_count) + ' more columns than expected')
            print(table_data)
        else:
            result.append(table_data)

        driver.find_element_by_class_name("ant-modal-close-x").click()
        time.sleep(1)
        
    print('Done')    
    print("---------------------")
    driver.find_element_by_css_selector(".anticon.anticon-right").click()
    time.sleep(2)
    if int(current_page) >= int(max_page):
        print('Finished all pages ...')
        print('Saving data ...')
        break    
    
file_name = "江苏排污权交易平台供需信息_" + time.strftime("%Y-%m-%d", time.localtime()) + ".csv"
DataFrame(result,columns=header).to_csv(file_name)
print(file_name + ' SAVED')   

110 pages in total.
---------------------
current page: 61/110
Done
---------------------
---------------------
current page: 62/110


KeyboardInterrupt: 